In [7]:
import numpy as np
import cv2

In [12]:
# SD Card sector information
headerSector =          1023 # Holds user settings to configure Miniscope and recording
configSector =          1024 # Holds final settings of the actual recording
dataStartSector =       1025 # Recording data starts here
sectorSize =            512

WRITE_KEY0 =				0x0D7CBA17
WRITE_KEY1 =				0x0D7CBA17
WRITE_KEY2 =				0x0D7CBA17
WRITE_KEY3 =				0x0D7CBA17

# SD Card Header Sector positions
HEADER_GAIN_POS =				4
HEADER_LED_POS =				5
HEADER_EWL_POS =				6
HEADER_RECORD_LENGTH_POS =  	7
HEADER_FRAME_RATE = 			8

# SD Card Config Sector positions
CONFIG_BLOCK_WIDTH_POS =			0
CONFIG_BLOCK_HEIGHT_POS	=   		1
CONFIG_BLOCK_FRAME_RATE_POS	=   	2
CONFIG_BLOCK_BUFFER_SIZE_POS =  	3

# Data Buffer Header positions
BUFFER_HEADER_BUFFER_LENGTH_POS =			0
BUFFER_HEADER_LINKED_LIST_POS = 			1
BUFFER_HEADER_FRAME_NUM_POS	=   			2
BUFFER_HEADER_BUFFER_COUNT_POS =			3
BUFFER_HEADER_FRAME_BUFFER_COUNT_POS =  	4
BUFFER_HEADER_TIMESTAMP_POS	=   			5
BUFFER_HEADER_DATA_LENGTH_POS = 			6

In [11]:
# variables that you can adjust
driveName = r"\\.\PhysicalDrive1"  # Change this to the correct drive

In [13]:
f = open(driveName, "rb")  # Open drive
f.seek(headerSector * sectorSize)  # Move to correct sector

PermissionError: [Errno 13] Permission denied: '\\\\.\\PhysicalDrive1'

In [ ]:
rawBytes = f.read(frameWidth * frameHeight)